In [1]:
from enum import Enum
from collections import defaultdict

In [2]:
# Utils
tadd = lambda a, b: tuple(map(sum, zip(a, b)))
assert tadd((1, 2), (3, 4)) == (4, 6)

```
If the current node is infected, it turns to its right. Otherwise, it turns to its left. (Turning is done in-place; the current node does not change.)
If the current node is clean, it becomes infected. Otherwise, it becomes cleaned. (This is done after the node is considered for the purposes of changing direction.)
The virus carrier moves forward one node in the direction it is facing.
```

In [3]:
class Direction(Enum):
    UP = 1
    DOWN = 2
    LEFT = 3
    RIGHT = 4
    
    def turn_left(self):
        left_map = {
            Direction.UP: Direction.LEFT,
            Direction.DOWN: Direction.RIGHT,
            Direction.LEFT: Direction.DOWN,
            Direction.RIGHT: Direction.UP,
        }
        return left_map[self]
    
    def turn_right(self):
        right_map = {
            Direction.UP: Direction.RIGHT,
            Direction.DOWN: Direction.LEFT,
            Direction.LEFT: Direction.UP,
            Direction.RIGHT: Direction.DOWN,
        }
        return right_map[self]
    
    def reverse(self):
        reverse_map = {
            Direction.UP: Direction.DOWN,
            Direction.DOWN: Direction.UP,
            Direction.LEFT: Direction.RIGHT,
            Direction.RIGHT: Direction.LEFT,
        }
        return reverse_map[self]
    
    def dont_turn(self):
        return self
    
    def move_forward(self, position):
        delta_map = {
            Direction.UP: (0, 1),
            Direction.DOWN: (0, -1),
            Direction.LEFT: (-1, 0),
            Direction.RIGHT: (1, 0),
        }
        return tadd(position, delta_map[self])


assert Direction.UP.turn_left() == Direction.LEFT
assert Direction.RIGHT.turn_right() == Direction.DOWN
assert Direction.UP.move_forward((1,1)) == (1,2)

```
Decide which way to turn based on the current node:
If it is clean, it turns left.
If it is weakened, it does not turn, and will continue moving in the same direction.
If it is infected, it turns right.
If it is flagged, it reverses direction, and will go back the way it came.
```

In [4]:
class NodeState(Enum):
    CLEAN = 1
    WEAKENED = 2
    INFECTED = 3
    FLAGGED = 4
    CLEAN_PART_1 = 5
    INFECTED_PART_1 = 6
    
    def transition(self):
        transition_map = {
            NodeState.CLEAN: NodeState.WEAKENED,
            NodeState.WEAKENED: NodeState.INFECTED,
            NodeState.INFECTED: NodeState.FLAGGED,
            NodeState.FLAGGED: NodeState.CLEAN,
            NodeState.CLEAN_PART_1: NodeState.INFECTED_PART_1,
            NodeState.INFECTED_PART_1: NodeState.CLEAN_PART_1,
        }
        return transition_map[self]
    
    def turn(self, direction):
        turn_map = {
            NodeState.CLEAN: direction.turn_left,
            NodeState.WEAKENED: direction.dont_turn,
            NodeState.INFECTED: direction.turn_right,
            NodeState.FLAGGED: direction.reverse,
            NodeState.CLEAN_PART_1: direction.turn_left,
            NodeState.INFECTED_PART_1: direction.turn_right,
        }
        return turn_map[self]()
        

In [5]:
class SporificaVirus(object):
    def __init__(self, cluster_map, default_state=NodeState.CLEAN):
        self.cluster_map = defaultdict(lambda:default_state, cluster_map)
        self.default_state = default_state
        self.direction = Direction.UP
        self.position = (0, 0)
        self.num_infections = 0
        
    def burst(self):
        node_state = self.cluster_map[self.position]        
        self.direction = node_state.turn(self.direction)
        new_state = node_state.transition()
        
        if new_state in (NodeState.INFECTED, NodeState.INFECTED_PART_1):
            self.num_infections += 1
        if new_state == self.default_state:
            del self.cluster_map[self.position]
        else:
            self.cluster_map[self.position] = new_state

        self.position = self.direction.move_forward(self.position)

Sample map:
```
..#
#..
...
```

In [6]:
def map_to_tuples(lines, infected_state=NodeState.INFECTED):
    infected_map = dict()
    x_start = -(len(lines[0]) // 2)
    y = len(lines) // 2
    
    for l in lines:
        x = x_start
        for c in list(l):
            if c == '#':
                infected_map[(x, y)] = infected_state
            x += 1
        y -= 1
    return infected_map

assert map_to_tuples(['..#', '#..', '...']) == {
    (1, 1): NodeState.INFECTED,
    (-1, 0): NodeState.INFECTED
}

In [7]:
virus = SporificaVirus(
    map_to_tuples(['..#', '#..', '...'], NodeState.INFECTED_PART_1),
    default_state=NodeState.CLEAN_PART_1
)
for r in range(70):
    virus.burst()
assert virus.num_infections == 41

In [8]:
virus = SporificaVirus(
    map_to_tuples(['..#', '#..', '...'], NodeState.INFECTED_PART_1),
    default_state=NodeState.CLEAN_PART_1
)
for r in range(10000):
    virus.burst()
assert virus.num_infections == 5587

In [9]:
with open('infection_map.txt') as fh:
    lines = fh.readlines()
lines = [l.strip() for l in lines]

In [10]:
virus = SporificaVirus(
    map_to_tuples(lines, NodeState.INFECTED_PART_1),
    default_state=NodeState.CLEAN_PART_1
)
for r in range(10000):
    virus.burst()
virus.num_infections

5240

## Part 2

In [11]:
virus = SporificaVirus(map_to_tuples(['..#', '#..', '...']))
for r in range(100):
    virus.burst()
assert virus.num_infections == 26

In [12]:
virus = SporificaVirus(map_to_tuples(lines))
for r in range(10000000):
    if r % 1000000 == 0:
        print('Progress', r)
    virus.burst()
virus.num_infections

Progress 0
Progress 1000000
Progress 2000000
Progress 3000000
Progress 4000000
Progress 5000000
Progress 6000000
Progress 7000000
Progress 8000000
Progress 9000000


2512144